# **Explicación del caso**

La empresa proporcionó un conjunto de sentencias judiciales históricas con el objetivo de construir un sistema capaz de:

- almacenar los fallos de forma estructurada,
- procesarlos y limpiarlos,
- generar representaciones vectoriales (embeddings),
- almacenarlos en una base de datos vectorial open source (ChromaDB),
- y permitir consultas semánticas para encontrar casos similares.

El propósito es permitir búsquedas “por significado” en lugar de palabras clave.

## **Supuestos**

1. Las fechas entregadas en el archivo están codificadas como números de Excel.
2. Las columnas confirmadas (idx, Relevancia, Providencia, etc.) no cambian.
3. No existe ruido significativo en la columna Sintesis, pero puede contener saltos de línea.
4. Para el prototipo, ChromaDB se ejecuta en memoria, sin persistencia.
5. El objetivo no es entrenar un modelo jurídico sino implementar un sistema semántico funcional.

### **Formas para resolver el caso y la opción tomada**

1. TF-IDF + Cosine Similarity: rápido, pero no captura significado profundo.
2. LLM embeddings con API externa: potente, pero dependiente de servicios externos.
3. Embeddings con SentenceTransformer + ChromaDB:
-   open source
-   liviano
-   funcional en Azure y local
-   recomendado explícitamente por la empresa

### **Futuros ajustes o mejoras**

- Persistencia en disco para reutilizar la DB entre ejecuciones.
- Embeddings legales especializados, como Legal-BERT.
- Chunking inteligente de textos largos.
- Dashboard con interfaz web (FastAPI + Streamlit).
- Re-ranking con un modelo LLM para precisión extra.

# **Prueba Técnica #2**

### **1. Traer el archivo sentencias_pasadas.xlsx a este espacio de trabajo y transformarlo a .csv**

In [1]:
from azureml.core import Workspace, Dataset, Datastore

ws = Workspace(subscription_id='35d4664b-ed82-4074-b51a-a96056a154ac',
               resource_group='grupopruebatecnica',
               workspace_name='WS-PruebaTecnica')

datastore = Datastore.get(ws, "workspaceblobstore")

dataset = Dataset.File.from_files(path=(datastore, "workspaceblobstore/sentencias_pasadas.xlsx"))

local_path = dataset.download('.', overwrite=True)
local_path

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/johanbolivarmosquera1/code/Users/johanbolivarmosquera/sentencias_pasadas.xlsx']

In [2]:
import zipfile
import xml.etree.ElementTree as ET
import csv

xlsx_path = "sentencias_pasadas.xlsx"
csv_path = "sentencias_pasadas.csv"

# --- Abrir XLSX como ZIP ---
with zipfile.ZipFile(xlsx_path) as z:
    sheet_xml = z.read('xl/worksheets/sheet1.xml')
    shared_xml = z.read('xl/sharedStrings.xml')

# --- Parsear sharedStrings.xml ---
shared_root = ET.fromstring(shared_xml)
ns = {'main': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}

shared_strings = []
for si in shared_root.findall('main:si', ns):
    text_elements = si.findall('.//main:t', ns)
    text = ''.join([t.text for t in text_elements if t.text])  
    shared_strings.append(text)

# Parsear sheet1.xml 
sheet_root = ET.fromstring(sheet_xml)

rows = []
for row in sheet_root.findall('.//main:row', ns):
    values = []
    for cell in row.findall('main:c', ns):
        cell_type = cell.get('t')  # si t="s", es shared string
        value = cell.find('main:v', ns)

        if value is None:
            values.append("")
        else:
            if cell_type == "s":
                # índice a sharedStrings.xml
                idx = int(value.text)
                values.append(shared_strings[idx])
            else:
                # valor directo
                values.append(value.text)
    rows.append(values)

# --- Guardar CSV ---
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

print("CSV generado correctamente:", csv_path)

CSV generado correctamente: sentencias_pasadas.csv


In [3]:
import pandas as pd

df = pd.read_csv("sentencias_pasadas.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7
0,#,Relevancia,Providencia,Tipo,Fecha Sentencia,Tema - subtema,resuelve,sintesis
1,1,966965,T-185/22,NaN,44712,NaN,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...
2,3,963168,T-356/21,NaN,44484,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...
3,5,956201,T-351/22,NaN,44841,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...
4,6,955889,T-246/21,NaN,44406,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...


### **2. Organizar los nombres de las columnas de la tabla y cambiar el formato de fecha**

In [4]:
import pandas as pd
import numpy as np

# CARGA DEL CSV 
df = pd.read_csv("sentencias_pasadas.csv", header=None)

# Asignar nombres de columnas manualmente
df.columns = ["idx", "Relevancia", "Providencia", "Tipo", "FechaSentencia",
              "Tema", "Resuelve", "Sintesis"]

# LIMPIEZA DE LA COLUMNA FECHA (Excel → dd/mm/yyyy)
# Mantener solo filas donde la fecha sea numérica
df = df[df["FechaSentencia"].astype(str).str.isnumeric()]

# Convertir a número
df["Fecha_num"] = pd.to_numeric(df["FechaSentencia"], errors="coerce")

# Convertir número Excel → fecha real
df["FechaSentencia"] = pd.to_datetime(
    df["Fecha_num"], unit="d", origin="1899-12-30"
)

# Formato dd/mm/yyyy
df["FechaSentencia"] = df["FechaSentencia"].dt.strftime("%d/%m/%Y")

# Eliminar columna auxiliar
df = df.drop(columns=["Fecha_num"])

# LIMPIEZA DE TEXTO 
for c in ["Providencia", "Tipo", "Tema", "Resuelve", "Sintesis"]:
    df[c] = df[c].astype(str).str.strip()

# 4. CREAR DOCUMENTO UNIFICADO (para embeddings)
def make_doc(row):
    parts = [
        f"Providencia: {row['Providencia']}",
        f"Tipo: {row['Tipo']}",
        f"Fecha: {row['FechaSentencia']}",
        f"Tema: {row['Tema']}",
        f"Resumen: {row['Sintesis']}"
    ]
    return "\n".join(parts)

df["documentText"] = df.apply(make_doc, axis=1)

# 5. REORDENAR COLUMNAS
column_order = [
    "idx", "Relevancia", "Providencia", "Tipo",
    "FechaSentencia", "Tema", "Resuelve", "Sintesis"
]

df = df[column_order + ["documentText"]]

# 6. MOSTRAR RESULTADO
print("Filas procesadas:", len(df))
df.head(10)

Filas procesadas: 329


,idx,Relevancia,Providencia,Tipo,FechaSentencia,Tema,Resuelve,Sintesis,documentText
1,1,966965,T-185/22,nan,31/05/2022,nan,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...,Providencia: T-185/22\nTipo: nan\nFecha: 31/05...
2,3,963168,T-356/21,nan,15/10/2021,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...,Providencia: T-356/21\nTipo: nan\nFecha: 15/10...
3,5,956201,T-351/22,nan,07/10/2022,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...,Providencia: T-351/22\nTipo: nan\nFecha: 07/10...
4,6,955889,T-246/21,nan,29/07/2021,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...,Providencia: T-246/21\nTipo: nan\nFecha: 29/07...
5,7,955787,T-245A/22,nan,01/07/2022,ACCION DE TUTELA-Inexistencia de hecho superad...,en nombre del pueblo y por mandato de la Const...,"El accionante, actuando en representación de s...",Providencia: T-245A/22\nTipo: nan\nFecha: 01/0...
6,8,954029,T-190/24,nan,23/05/2024,nan,RESUELVE PRIMERO. CONFIRMAR la decisión profer...,El actor solicitó la protección de los derecho...,Providencia: T-190/24\nTipo: nan\nFecha: 23/05...
7,9,947406,T-394/24,nan,19/09/2024,nan,RESUELVE PRIMERO. CONFIRMAR la Sentencia del 1...,La presente acción de tutela fue formulada por...,Providencia: T-394/24\nTipo: nan\nFecha: 19/09...
8,10,945053,T-155/19,nan,04/04/2019,DERECHO A LA INTIMIDAD-Alcance y contenido DER...,en nombre del pueblo y por mandato de la Const...,"En el presente caso, se atribuye la vulneració...",Providencia: T-155/19\nTipo: nan\nFecha: 04/04...
9,11,943156,T-339/22,nan,28/09/2022,CARENCIA ACTUAL DE OBJETO POR DAÑO CONSUMADO-D...,en nombre del pueblo y por mandato de la Const...,La actora adujo que la persona accionada vulne...,Providencia: T-339/22\nTipo: nan\nFecha: 28/09...
10,14,939.17,T-281/21,nan,23/08/2021,ACCION DE TUTELA CONTRA PARTICULARES-Solicitud...,en nombre del pueblo y por mandato de la Const...,En este caso se instauró la acción de tutela b...,Providencia: T-281/21\nTipo: nan\nFecha: 23/08...


In [9]:
# Sobrescribir el CSV original
df.to_csv("sentencias_pasadas.csv", index=False)

##### **Instalar ChromaDB**

In [20]:
%pip install "opentelemetry-sdk==1.23.0" "opentelemetry-api==1.23.0" "opentelemetry-proto==1.23.0" --force-reinstall
%pip install chromadb sentence-transformers

  Attempting uninstall: zipp
    Found existing installation: zipp 3.23.0
    Uninstalling zipp-3.23.0:
      Successfully uninstalled zipp-3.23.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.14.1
    Uninstalling typing_extensions-4.14.1:
      Successfully uninstalled typing_extensions-4.14.1━━━━━━━━━━━  2/10 [typing-extensions]
  Attempting uninstall: protobuf90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/10 [typing-extensions]
    Found existing installation: protobuf 6.31.1━━━━━━━━━━━━━━  2/10 [typing-extensions]
    Uninstalling protobuf-6.31.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/10 [typing-extensions]
      Successfully uninstalled protobuf-6.31.1━━━━━━━━━━━━━━━━  2/10 [typing-extensions]
  Attempting uninstall: opentelemetry-semantic-conventions━━━━  2/10 [typing-extensions]
    Found existin

In [5]:
from sentence_transformers import SentenceTransformer
import chromadb

# Modelo
model = SentenceTransformer("all-MiniLM-L6-v2")

# Cliente Chroma (versión lite)
client = chromadb.Client()

# Colección con recuperación o creación
collection = client.get_or_create_collection(
    name="sentencias_legal",
    metadata={"hnsw:space": "cosine"}
)

# Insertar documentos
for idx, row in df.iterrows():
    collection.add(
        ids=[str(idx)],
        documents=[row["documentText"]],
        embeddings=[model.encode(row["documentText"]).tolist()],
        metadatas=[{
            "Relevancia": row["Relevancia"],
            "Providencia": row["Providencia"],
            "Tipo": row["Tipo"],
            "FechaSentencia": row["FechaSentencia"],
            "Tema": row["Tema"],
            "Resuelve": row["Resuelve"],
            "Sintesis": row["Sintesis"]
        }]
    )


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar el modelo para generar embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Función para buscar los casos más similares a un query
def buscar_casos(query, top_k=3):
    # Generar embedding del query
    query_embedding = model.encode(query).tolist()
    
    # Buscar en Chroma
    resultados = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    # Validar si hay resultados
    if (len(resultados["documents"]) == 0
        or resultados["documents"][0] is None):
        return ["No se encontraron resultados similares."]
    
    docs = resultados['documents'][0]
    metadatos = resultados['metadatas'][0]
    
    respuestas = []
    for doc, meta in zip(docs, metadatos):
        texto = (
            f"Providencia: {meta['Providencia']}\n"
            f"Tipo: {meta['Tipo']}\n"
            f"Fecha: {meta['FechaSentencia']}\n"
            f"Tema: {meta['Tema']}\n"
            f"Resumen: {meta['Sintesis']}"
        )
        respuestas.append(texto)

    return respuestas

In [8]:
# Tres demandas sobre redes sociales
print("=== Tres demandas sobre redes sociales ===")
casos_redes = buscar_casos("demandas relacionadas con redes sociales", top_k=3)
for i, c in enumerate(casos_redes, 1):
    print(f"\nCaso {i}:\n{c}")

# Caso de acoso escolar
print("\n=== Caso de acoso escolar ===")
casos_acoso = buscar_casos("acoso escolar", top_k=1)
for c in casos_acoso:
    print(c)

# Casos relacionados con PIAR
print("\n=== Casos relacionados con PIAR ===")
casos_piar = buscar_casos("PIAR", top_k=3)
for i, c in enumerate(casos_piar, 1):
    print(f"\nCaso {i}:\n{c}")

=== Tres demandas sobre redes sociales ===

Caso 1:
Providencia: T-087/23
Tipo: nan
Fecha: 28/03/2023
Tema: nan
Resumen: Las accionantes son periodistas que consideran haber sido víctimas de diferentes ataques en línea a través de la red social Twitter, de naturaleza misógina y de contenido sexualizado, que buscan infantilizar su oficio y censurarlas. En sede de tutela cuestionaron a la Dirección Nacional Electoral por no adoptar ninguna medida para hacer cesar la violencia, sancionar a los responsables y prevenirla. Así mismo, criticaron el hecho de que los partidos políticos y/o movimientos ciudadanos se hubieran favorecido de las agresiones, al alentarlas o tolerarlas. Se analiza temática relacionada con: 1°. La violencia en línea contra mujeres periodistas. 2°. El derecho a la libertad de expresión en las redes sociales y, 3°. El régimen sancionatorio de los actores, partidos y movimientos políticos. A pesar de no encontrar demostrada la vulneración de los derechos fundamentales de